# 实例描述
将cifar卷积代码中的每层卷积结果进行反卷积并输出，通过Tensorboard观察其结果。

1. 替换Maxpool池化函数

In [2]:
import tensorflow as tf

/usr/local/Cellar/python/3.6.5/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

def avg_pool6x6(x):
    return tf.nn.avg_pool(x,ksize=[1,6,6,1],strides=[1,6,6,1],padding='SAME')

def max_pool_with_argmax(net,stride):
    _,mask = tf.nn.max_pool_with_argmax(net,ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
    mask=tf.stop_gradient(mask)
    net = tf.nn.max_pool(net,ksize=[1,stride,stride,1],strides=[1,stride,stride,1],padding='SAME')
    return net,mask

# 反最大池化
def unpool(net,mask,stride):
    
    ksize = [1, stride, stride, 1]
    input_shape = net.get_shape().as_list()
    #  calculation new shape
    output_shape = (input_shape[0], input_shape[1] * ksize[1], input_shape[2] * ksize[2], input_shape[3])
    # calculation indices for batch, height, width and feature maps
    one_like_mask = tf.ones_like(mask)
    batch_range = tf.reshape(tf.range(output_shape[0], dtype=tf.int64), shape=[input_shape[0], 1, 1, 1])
    b = one_like_mask * batch_range
    y = mask // (output_shape[2] * output_shape[3])
    x = mask % (output_shape[2] * output_shape[3]) // output_shape[3]
    feature_range = tf.range(output_shape[3], dtype=tf.int64)
    f = one_like_mask * feature_range
    # transpose indices & reshape update values to one dimension
    updates_size = tf.size(net)
    indices = tf.transpose(tf.reshape(tf.stack([b, y, x, f]), [4, updates_size]))
    values = tf.reshape(net, [updates_size])
    ret = tf.scatter_nd(indices, values, output_shape)
    return ret

In [6]:
x = tf.placeholder(tf.float32,[None,24,24,3]) 
y = tf.placeholder(tf.float32,[None,10]) 

W_conv1 = weight_variable([5,5,3,64])
b_conv1 = bias_variable([64])

x_image = tf.reshape(x,[-1,24,24,3])

h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
h_pool1,mask1 = max_pool_with_argmax(h_conv1,2)

W_conv2 = weight_variable([5,5,64,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2,mask = max_pool_with_argmax(h_conv2,2)
print("h_pool2_shape:",h_pool2.shape)

# W_conv3 = weight_variable([5,5,64,10])
# b_conv3 = bias_variable([10])
# h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3) + b_conv3)

# nt_hpool3 = avg_pool6x6(h_conv3)
# nt_hpool3_flat = tf.reshape(nt_hpool3,[-1,10])
# y_conv = tf.nn.softmax(nt_hpool3_flat)

# cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))

# train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

h_pool2_shape: (?, 6, 6, 64)


> 上面最后一行是将h_pool2的形状打印出来，这也是在组建网络结构时常用的一种调试方法。**反卷积和反池化对形状都很敏感**(尤其层数太多时)。

2.反卷积第二层卷积结果
以第二池化输出的变量h_pool2为开始部分，沿着h_pool2生成的方式反向操作一层一层推导，直到生成原始图。![](imgs/21_deconvolution.png)
上半部分是h_pool2卷积的过程，下半部分为反卷积过程。因为在卷积过程中，每个卷积后都要加上权重b，所以在反卷积过程中就要将b减去。由于Relu函数基本上是恒等变化（出了小于0的部分），所以在反向时不需要可逆操作，可直接略去。

In [8]:
t_conv2 = unpool(h_pool2,mask,2) #(128,12,12,64)

t_pool1 = tf.nn.conv2d_transpose(t_conv2 - b_conv2,W_conv2,h_pool1.shape,[1,1,1,1]) # (128,24,24,64)
print(t_conv2.shape,h_pool1.shape,t_pool1.shape)
t_conv1 = unpool(t_pool1,mask1,2)
t_x_image = tf.nn.conv2d_transpose(t_conv1-b_conv1,W_conv1,x_image.shape,[1,1,1,1])

3.反卷积第一层卷积结果

In [10]:
# 第一层卷积还原
t1_conv1 = unpool(h_pool1,mask1,2)
t1_x_image = tf.nn.conv2d_transpose(t1_conv1-b_conv1,W_conv1,x_image.shape,[1,1,1,1])

4.合并还原结果，并输出给tensorboard输出

In [ ]:
# 生成最终图像
stitched_decodings = tf.concat((x_image,t1_x_image,t_x_image),axis=2)
decoding_summary_op = tf.summary.image('source/cifar',stitched_decodings)

5.session中写入log

In [ ]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_conv)) +(tf.nn.l2_loss(W_conv1)+tf.nn.l2_loss(W_conv2)+tf.nn.l2_loss(W_conv3))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))


sess = tf.Session()
sess.run(tf.global_variables_initializer())
summary_writer = tf.summary.FileWriter('./log/', sess.graph)

tf.train.start_queue_runners(sess=sess)

for i in range(15000):#20000
  image_batch, label_batch = sess.run([images_train, labels_train])
  label_b = np.eye(10,dtype=float)[label_batch] #one hot
  
  train_step.run(feed_dict={x:image_batch, y: label_b},session=sess)
  #_, decoding_summary = sess.run([train_step, decoding_summary_op],feed_dict={x:image_batch, y: label_b})
  
  if i%200 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:image_batch, y: label_b},session=sess)
    print( "step %d, training accuracy %g"%(i, train_accuracy))
    print("cross_entropy",cross_entropy.eval(feed_dict={x:image_batch, y: label_b},session=sess))
    #summary_writer.add_summary(decoding_summary)


image_batch, label_batch = sess.run([images_test, labels_test])
label_b = np.eye(10,dtype=float)[label_batch]#one hot
print ("finished！ test accuracy %g"%accuracy.eval(feed_dict={
     x:image_batch, y: label_b},session=sess))
decoding_summary = sess.run(decoding_summary_op,feed_dict={x:image_batch, y: label_b})
summary_writer.add_summary(decoding_summary)

6.Tensorboard中查看![](imgs/21_deconvolution_image.png)

第一幅是原始图片，不清晰的原因在于cifar10_input.inputs代码中，将图片做了归一化（变成-1~1之间的数）。第二幅是第一个卷积层还原的图片，第三幅是最后一个卷积层还原的图片。最后的卷积输出对图像的主要特征响应更强烈。

为了让图片看的更清晰，可以先去掉归一化的操作。![](imgs/21_convolution_image.png)

与归一化的效果对比，显然**归一化的图片卷积效果特征会更加明显**，这也是为什么要做归一化的原因。